In [109]:
import numpy as np
import random
from math import floor

In [110]:

MIN_SIZE = 2
MAX_SIZE = 10
DEFAULT_SIZE = 3


In [111]:
#Function to assert the builded grid is solvable
def is_solvable(grid):
    flat_grid = grid.flatten() 
    
    inversions = 0
    for i in range(len(flat_grid)):
        for j in range(i + 1, len(flat_grid)):
            if flat_grid[j] and flat_grid[i] and flat_grid[i] > flat_grid[j]:
                inversions += 1
    return inversions % 2 == 0

def build_line(start_symb, stop_symb, sep_symb, line):
    return '\n%s%s%s\n' % (start_symb, sep_symb.join(line), stop_symb)

In [112]:
# State : Correspond a la grid
# Actions: Dé deplacer :
#0: dé en haut
#1: dé a gauche
#2: dé en bas
#3: dé à droite

class Grid :
    def __init__(self,size=DEFAULT_SIZE) -> None:
        assert size>=MIN_SIZE and size <= MAX_SIZE
        self.size=size

        while True:
            grid=list(range(size**2))
            random.shuffle(grid)
            grid=np.asarray(grid).reshape(size, size)
            if is_solvable(grid):break
        self.state=grid


    def take_action(self,action):
      i, j = np.where(self.state == 0)
      if len(i) > 0 and len(j) > 0:
        i = i[0]
        j = j[0]
        if action == 0:
            if i > 0:
                self.state[i][j], self.state[i-1][j] = self.state[i-1][j], self.state[i][j]
        elif action == 2:
            if i < self.size-1:
                self.state[i][j], self.state[i+1][j] = self.state[i+1][j], self.state[i][j]
        elif action ==1:
            if j > 0:
                self.state[i][j], self.state[i][j-1] = self.state[i][j-1], self.state[i][j]
        elif action == 3:
            if j < self.size-1:
                self.state[i][j], self.state[i][j+1] = self.state[i][j+1], self.state[i][j]

    def is_finish(self):
        ended_grid=[i for i in range(1,self.size**2)]
        ended_grid.append(0)
        return np.array_equal(
            self.state,
            np.asarray(ended_grid).reshape(self.size,self.size)
            )
    
    def get_possible_actions(self)-> list[int]:
        """return the possible actions"""
        actions=[i for i in range(0,4)]
        pos_empty=self.get_empty_position()
        if pos_empty[0]==0 : actions.remove(0)
        elif pos_empty[0]==(self.size-1):actions.remove(2)
        if pos_empty[1]==0:actions.remove(1)
        elif pos_empty[1]==(self.size-1):actions.remove(3)
        return  actions
    
    def get_empty_position(self)->tuple:
        """Return the position od the tuple"""
        return floor(np.argmin(self.state)/self.size),np.argmin(self.state)%self.size

    def get_good_place(self):
      ended_grid=[i for i in range(1,self.size**2)]
      ended_grid.append(0)
      return (np.asarray(ended_grid).reshape(self.size,self.size) == self.state ).sum()

    def __str__(self) -> str:
        """Renderer"""
        tile_line = np.full(self.size, '─' * 4).tolist()
        horizontal_line = build_line('├', '┤', '┼', tile_line)
        first_horizontal_line = build_line('┌', '┐', '┬', tile_line)
        last_horizontal_line = build_line('└', '┘', '┴', tile_line)
        grid_to_show = first_horizontal_line
        for count, row in enumerate(self.state):
            grid_to_show += '│'
            for tile in row:
                if tile == 0:
                    tile = '  '
                grid_to_show += ' %s │' % '{0:>2}'.format(tile)
            if not count == self.size - 1:
                grid_to_show += horizontal_line
        grid_to_show += last_horizontal_line
        grid_to_show+=f"\n Possible actions: {self.get_possible_actions()}"
        return grid_to_show
        
    

In [113]:
grid=Grid(size=3)
print(grid)
grid.is_finish()


┌────┬────┬────┐
│  7 │  4 │  5 │
├────┼────┼────┤
│  2 │    │  3 │
├────┼────┼────┤
│  8 │  1 │  6 │
└────┴────┴────┘

 Possible actions: [0, 1, 2, 3]


False

In [114]:
class Game:
    def __init__(self) -> None:
        self.grid=Grid()
        self.end=False
        self.round=0
        self.reward=0
        self.sum_reward=0
        self.actions=[]# list of all actions done in the game
        self.last_good=0
        self.number_good_place=0

    def play_round(self):
        """function_agent : fonction de choix de l'agent"""
        reward=-1
        action=random.choice(self.grid.get_possible_actions())
        #action=np.random.randint(0,3)
        self.grid.take_action(action)
        if self.grid.is_finish():
          self.end = True
          reward += 50
        print(f"Number of well placed : {self.grid.get_good_place()}")
        if self.grid.get_good_place() == self.number_good_place:
          reward -= self.last_good # more we dont increase the number of well positioned stuff more reward we loose
          self.last_good += 1
        elif self.grid.get_good_place() < self.number_good_place:
          reward -= 5
        else:
          reward+=self.grid.get_good_place()
          self.number_good_place = self.grid.get_good_place()
        return reward,self.grid.state,action


    def play_game(self):
        while not self.end and self.round < 1000:
            reward,state,action=self.play_round()
            self.reward=reward
            self.sum_reward+=reward
            self.actions.append(action)
            self.round+=1
            print(f"round: {self.round}")
            print(f"action: {action}")
            print(f"Sum of reward: {self.sum_reward}")
            print(self.grid)

    def __str__(self):
        game_to_show=f"---- {self.round} ----------\n"
        game_to_show+=self.grid.__str__()
        game_to_show+=f"\nReward Round{self.reward} \nReward cum:{self.sum_reward}"
        return game_to_show




In [115]:
gg=Game()

In [ ]:
gg.play_game()

In [ ]:
import numpy as np

class Qlearning:
    def __init__(self, actions, alpha=0.1, gamma=0.9, epsilon=0.1,size=2):
        self.Q = {}
        self.actions = actions
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.size=size**2

    def get_Q(self, state, action):
        if tuple(state.reshape(self.size)) not in self.Q:
            self.Q[tuple(state.reshape(self.size))] = np.zeros(len(self.actions))
        return self.Q[tuple(state.reshape(self.size))][action]

    def choose_action(self, state,possible_actions):
        if np.random.uniform() < self.epsilon:
            # Choose a random action
            return np.random.choice(possible_actions)
        else:
            # Choose the action with the highest Q-value
            q_values = [self.get_Q(state, a) for a in possible_actions]
            max_q = max(q_values)
            if q_values.count(max_q) > 1:
                # Multiple actions have the same highest Q-value, choose one randomly
                
                return possible_actions[np.random.choice(np.flatnonzero(np.array(q_values) == np.array(q_values).max()))]
            else:
                return possible_actions[np.argmax(q_values)]

    def update(self, state, action, reward, next_state):
        old_Q = self.get_Q(state, action)
        next_max_Q = max([self.get_Q(next_state, a) for a in self.actions])
        new_Q = old_Q + self.alpha * (reward + self.gamma * next_max_Q - old_Q)
        #print(action,reward)
        self.Q[tuple(state.reshape(self.size))][action] = new_Q
        #print(self.Q)


In [ ]:
size=3
qlearner = Qlearning(actions=[0, 1, 2, 3],size=size)

In [ ]:
qlearner.epsilon=0.7

nb_game = 1000
win=0
for i in range(nb_game):
  game = Game(size=size)
  round=0
  res=game.play_game(qlearner,25000)
  if res:
    win+=1
  print(i,res)